<a href="https://colab.research.google.com/github/chavgova/My-AI/blob/master/Text_Emotion_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SPEECH TO TEXT**

In [2]:
!pip install SpeechRecognition
!pip install pyttsx3

     |████████████████████████████████| 32.8MB 122kB/s 


In [3]:
!pip install ffmpeg-python

In [4]:
!sudo apt-get install portaudio19-dev python-pyaudio python3-pyaudio

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libportaudio2 libportaudiocpp0
Suggested packages:
  portaudio19-doc python-pyaudio-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev python-pyaudio
  python3-pyaudio
0 upgraded, 5 newly installed, 0 to remove and 11 not upgraded.
Need to get 232 kB of archives.
After this operation, 1,108 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudiocpp0 amd64 19.6.0-1 [15.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 portaudio19-dev amd64 19.6.0-1 [104 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python-pyaudio amd64 0.2.11-1build2 [24.1 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic/universe a

In [5]:
"""
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

In [6]:
audio, sr = get_audio()

In [7]:
type(audio)

numpy.ndarray

In [8]:
from scipy.io.wavfile import write

samplerate = 44100; fs = 100
t = np.linspace(0., 1., samplerate)
amplitude = np.iinfo(np.int16).max
data = amplitude * np.sin(2. * np.pi * fs * t)
write("/content/example_new.wav", samplerate, audio)

In [9]:
import speech_recognition as sr

r = sr.Recognizer()
file = sr.AudioFile("/content/example_new.wav")
# for transcripting audio
with file as source:
    audio = r.record(source)
#  Speech recognition using Google Speech Recognition
try:
    recog = r.recognize_google(audio, language = 'en-US')
    # for testing purposes, we're just using the default API key
    # to use another API key, use r.recognize_google(audio, key="GOOGLE_SPEECH_RECOGNITION_API_KEY")
    # instead of `r.recognize_google(audio)`` 

    print("You said: " + recog)
except sr.UnknownValueError:
    print("Google Speech Recognition could not understand audio")
except sr.RequestError as e:
    print("Could not request results from Google Speech Recognition service; {0}".format(e)) 

You said: this place is abandoned


# SENTENCES

In [1]:
!pip install --upgrade deepsegment

     |████████████████████████████████| 378kB 5.9MB/s 
     |████████████████████████████████| 51kB 4.5MB/s 
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


In [5]:
from deepsegment import DeepSegment

segmenter = DeepSegment('en')
segmenter.segment('I am Batman i live in gotham')

AttributeError: ignored

In [2]:
import deepsegment

deepsegment.download('eng_fra_ita')

Using TensorFlow backend.


AttributeError: ignored

In [7]:
!pip install nnsplit

     |████████████████████████████████| 2.4MB 2.8MB/s 


In [8]:
from nnsplit import NNSplit

splitter = NNSplit.load("en")

# returns `Split` objects
splits = splitter.split(["This is a test this is another test."])[0]

# a `Split` can be iterated over to yield smaller splits or stringified with `str(...)`.
for sentence in splits:
   print(sentence)

AttributeError: ignored

# **EMOTIONS** - via word dictionary

In [10]:
import pandas as pd

In [11]:
!mkdir -p data
!wget -nc https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/upshot-trump-emolex/data/NRC-emotion-lexicon-wordlevel-alphabetized-v0.92.txt -P data

--2020-09-23 20:33:32--  https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/upshot-trump-emolex/data/NRC-emotion-lexicon-wordlevel-alphabetized-v0.92.txt
Resolving nyc3.digitaloceanspaces.com (nyc3.digitaloceanspaces.com)... 162.243.189.2
Connecting to nyc3.digitaloceanspaces.com (nyc3.digitaloceanspaces.com)|162.243.189.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2581050 (2.5M) [text/plain]
Saving to: ‘data/NRC-emotion-lexicon-wordlevel-alphabetized-v0.92.txt’

NRC-emotion-lexicon 100%[===================>]   2.46M  11.2MB/s    in 0.2s    

2020-09-23 20:33:32 (11.2 MB/s) - ‘data/NRC-emotion-lexicon-wordlevel-alphabetized-v0.92.txt’ saved [2581050/2581050]



In [12]:
filepath = "/content/data/NRC-emotion-lexicon-wordlevel-alphabetized-v0.92.txt"
emolex_df = pd.read_csv(filepath,  names=["word", "emotion", "association"], skiprows=45, sep='\t', keep_default_na=False)
emolex_df.head(12)

,word,emotion,association
0,aback,anger,0
1,aback,anticipation,0
2,aback,disgust,0
3,aback,fear,0
4,aback,joy,0
5,aback,negative,0
6,aback,positive,0
7,aback,sadness,0
8,aback,surprise,0
9,aback,trust,0


In [13]:
# all emotions
print(emolex_df.emotion.unique())

print(emolex_df.emotion.value_counts())
# How many words does each emotion have ------- We're only going to care about "is associated."
print(emolex_df[emolex_df.association == 1].emotion.value_counts()) 

['anger' 'anticipation' 'disgust' 'fear' 'joy' 'negative' 'positive'
 'sadness' 'surprise' 'trust']
fear            14182
sadness         14182
trust           14182
joy             14182
negative        14182
surprise        14182
anticipation    14182
positive        14182
disgust         14182
anger           14182
Name: emotion, dtype: int64
negative        3324
positive        2312
fear            1476
anger           1247
trust           1231
sadness         1191
disgust         1058
anticipation     839
joy              689
surprise         534
Name: emotion, dtype: int64


In [14]:
emolex_words = emolex_df.pivot(index='word', columns='emotion', values='association').reset_index()
emolex_words.head()

emotion,word,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust
0,aback,0,0,0,0,0,0,0,0,0,0
1,abacus,0,0,0,0,0,0,0,0,0,1
2,abandon,0,0,0,1,0,1,0,1,0,0
3,abandoned,1,0,0,1,0,1,0,1,0,0
4,abandonment,1,0,0,1,0,1,0,1,1,0


In [15]:
# only angry words
emolex_df[(emolex_df.association == 1) & (emolex_df.emotion == 'anger')].word

30          abandoned
40        abandonment
170             abhor
180         abhorrent
270           abolish
             ...     
141220       wrongful
141230        wrongly
141470           yell
141500           yelp
141640          youth
Name: word, Length: 1247, dtype: object

In [16]:
anger_words = str(emolex_df[(emolex_df.association == 1) & (emolex_df.emotion == 'anger')].word)
anticipation_words = str(emolex_df[(emolex_df.association == 1) & (emolex_df.emotion == 'anticipation')].word)
disgust_words = str(emolex_df[(emolex_df.association == 1) & (emolex_df.emotion == 'disgust')].word)
fear_words = str(emolex_df[(emolex_df.association == 1) & (emolex_df.emotion == 'fear')].word)
joy_words = str(emolex_df[(emolex_df.association == 1) & (emolex_df.emotion == 'joy')].word)
negative_words = str(emolex_df[(emolex_df.association == 1) & (emolex_df.emotion == 'negative')].word)
positive_words = str(emolex_df[(emolex_df.association == 1) & (emolex_df.emotion == 'positive')].word)
sadness_words = str(emolex_df[(emolex_df.association == 1) & (emolex_df.emotion == 'sadness')].word)
surprise_words = str(emolex_df[(emolex_df.association == 1) & (emolex_df.emotion == 'surprise')].word)
trust_words = str(emolex_df[(emolex_df.association == 1) & (emolex_df.emotion == 'trust')].word)

def sentence_to_list(lst): 
    return (lst[0].split()) 

text = ['this place is an abandoned park']
# text = ['this place is a park']

sentence_word_list = sentence_to_list(text)

for word in sentence_word_list:
  current_word = ' ' + word

  if current_word in anger_words:
    print('anger')
  if current_word in anticipation_words:
    print('anticipation')
  if current_word in disgust_words:
    print('disgust')
  if current_word in fear_words:
    print('fear')
  if current_word in joy_words:
    print('joy')
  if current_word in negative_words:
    print('negative')
  if current_word in positive_words:
    print('positive')
  if current_word in sadness_words:
    print('sadness')
  if current_word in surprise_words:
    print('surprise')
  if current_word in trust_words:
    print('trust')

anger
fear
negative
sadness


# **TEXT TO SPEECH**

In [ ]:
!pip install gTTS

  Created wheel for gtts-token: filename=gTTS_token-1.1.3-cp36-none-any.whl size=4096 sha256=30fdc75c450abbcbec7f13a83100413f0945a4ce279eb18e92dceb13cc908c8b
  Stored in directory: /root/.cache/pip/wheels/dd/11/61/33f7e51bf545e910552b2255eead2a7cd8ef54064b46dceb34
Successfully built gtts-token


In [ ]:
from gtts import gTTS   ######## NO VOICES

from IPython.display import Audio
tts = gTTS('Tell me how are you madam')
tts.save('1.wav')
sound_file = '1.wav'
Audio(sound_file, autoplay=True)

In [ ]:
!pip install pyttsx3

In [19]:
!pwd  # directory
import pyttsx3
print(dir(pyttsx3))

/content
['Engine', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_activeEngines', 'driver', 'drivers', 'engine', 'init', 'six', 'speak', 'weakref']


In [6]:
import pyttsx3 
import os
os.chdir('/usr/local/lib/python3.6/dist-packages (2.90)')

engine = pyttsx3.init()
text = "Python is a great programming language"
engine.say(text)

engine.runAndWait()

FileNotFoundError: ignored

In [ ]:
voices = engine.getProperty("voices")
print(voices)

NameError: ignored

In [ ]:
engine.setProperty("voice", voices[1].id)
engine.say(text)
engine.runAndWait()

# **TEXTBLOB**

In [ ]:
from textblob import TextBlob
import nltk
nltk.download('brown')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
thing = TextBlob("Python is a high-level, general-purpose programming language.")
print()
print(thing.noun_phrases)
print(thing.tags)
print(thing.words) # Tokenization


['python']
[('Python', 'NNP'), ('is', 'VBZ'), ('a', 'DT'), ('high-level', 'JJ'), ('general-purpose', 'JJ'), ('programming', 'NN'), ('language', 'NN')]
['Python', 'is', 'a', 'high-level', 'general-purpose', 'programming', 'language']


In [ ]:
testimonial = TextBlob("Textblob is amazingly simple to use. What great fun!")
print(testimonial.sentiment)
print(testimonial.sentiment.polarity)

Sentiment(polarity=0.39166666666666666, subjectivity=0.4357142857142857)
0.39166666666666666


In [ ]:
zen = TextBlob("Beautiful is better than ugly. " "Explicit is better than implicit. " "Simple is better than complex.")
print(zen.sentences)

for sentence in zen.sentences:
  print(sentence.sentiment)

[Sentence("Beautiful is better than ugly."), Sentence("Explicit is better than implicit."), Sentence("Simple is better than complex.")]
Sentiment(polarity=0.2166666666666667, subjectivity=0.8333333333333334)
Sentiment(polarity=0.5, subjectivity=0.5)
Sentiment(polarity=0.06666666666666667, subjectivity=0.41904761904761906)


In [ ]:
from textblob import Word

Word("kitten").definitions

['young domestic cat', 'have kittens']

# **SPACY**

In [50]:
!python -m spacy download en_core_web_sm
!python -m spacy download en

import spacy
from spacy import displacy
nlp = spacy.load("en_core_web_sm")

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
doc = nlp("This project is driving me crazy!")
print([(w.text, w.pos_) for w in doc])

[('This', 'DET'), ('project', 'NOUN'), ('is', 'AUX'), ('driving', 'VERB'), ('me', 'PRON'), ('crazy', 'ADJ'), ('!', 'PUNCT')]


In [59]:
doc = nlp("that is Google")
for ent in doc.ents:
  print(ent.text, ent.label_)

from spacy import displacy 
displacy.render(doc, jupyter = True, style = "ent")

Google ORG


In [ ]:
displacy.render(doc, jupyter = True, style = "dep")

In [ ]:
for token in doc:
   print (token, token.idx)

This 0
Python 5
project 12
is 20
driving 23
me 31
crazy 34
! 39


In [ ]:
for token in doc:
    # Print the token and its part-of-speech tag
    print(token.text, "-->", token.pos_)

This --> DET
Python --> PROPN
project --> NOUN
is --> AUX
driving --> VERB
me --> PRON
crazy --> ADJ
! --> PUNCT


In [ ]:
conference_help_text = ('Prime Minister Narendra Modi on Tuesday announced the 266 billion dollars package for the India to fight against the coronavirus pandemic.')
conference_help_doc = nlp(conference_help_text)
for token in conference_help_doc:
    print (token, ' -> ', token.lemma_)

Prime  ->  Prime
Minister  ->  Minister
Narendra  ->  Narendra
Modi  ->  Modi
on  ->  on
Tuesday  ->  Tuesday
announced  ->  announce
the  ->  the
266  ->  266
billion  ->  billion
dollars  ->  dollar
package  ->  package
for  ->  for
the  ->  the
India  ->  India
to  ->  to
fight  ->  fight
against  ->  against
the  ->  the
coronavirus  ->  coronavirus
pandemic  ->  pandemic
.  ->  .


# **VADER**

https://medium.com/analytics-vidhya/simplifying-social-media-sentiment-analysis-using-vader-in-python-f9e6ec6fc52f

In [ ]:
pip install vaderSentiment

     |████████████████████████████████| 133kB 2.6MB/s 


In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    print("{:-<2} {}".format(sentence, str(score)))

In [ ]:
sentiment_analyzer_scores("He bullied me!")

He bullied me! {'neg': 0.687, 'neu': 0.313, 'pos': 0.0, 'compound': -0.6588}
